In [111]:
pip install pandera -q

Note: you may need to restart the kernel to use updated packages.


In [112]:
import pandas as pd
import pandera as pa

In [113]:
df = pd.read_csv("https://raw.githubusercontent.com/ftiosso/dio-curso-etl/main/ocorrencia_2010_2020.csv",
                 sep=";", parse_dates=["ocorrencia_dia"], dayfirst=True)
df.head(5)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [114]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo":pa.Column(pa.String),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [115]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


In [116]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [117]:
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5747    12
5748    12
5749    12
5750    12
5751    12
Name: ocorrencia_dia, Length: 5752, dtype: int64

## AGORA IREMOS FAZER A LIMPEZA DE DADOS

In [118]:
#selecionar algo especifico
df.loc[1,'ocorrencia_cidade']

'BELÉM'

In [119]:
#selecionar duas linhas
df.loc[1:3]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0


In [120]:
#selecionar duas linhas
df.loc[[10,40]]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,****,2010-01-10,21:30:00,0
40,39158,39158,INCIDENTE,BELÉM,PA,****,2010-01-28,16:00:00,0


In [121]:
#selecionar uma coluna inteira
df.loc[:,'ocorrencia_cidade']

0           RIO DE JANEIRO
1                    BELÉM
2           RIO DE JANEIRO
3       LUCAS DO RIO VERDE
4                  PELOTAS
               ...        
5747              CAMPINAS
5748     LAGOA DA CONFUSÃO
5749        RIO DE JANEIRO
5750             VICENTINA
5751        RIO DE JANEIRO
Name: ocorrencia_cidade, Length: 5752, dtype: object

In [122]:
#verificar se ela nao tem valores repetidos
df.codigo_ocorrencia.is_unique

True

In [123]:
#selecionar a coluna como index
#df.set_index('codigo_ocorrencia', inplace=True)

In [124]:
#selecionar pelo index
#df.loc[40324]

In [125]:
#resetar index
#df.reset_index(drop=True)


In [126]:
df.loc[0, 'ocorrencia_aerodromo'] = ''

In [127]:
#criar bkp coluna
df['ocorrencia_uf_bkp'] = df.ocorrencia_uf

In [131]:
#alterar uma coluna com base em outra
df.loc[df.ocorrencia_uf == 'SP', ['ocorrencia_classificacao']] = 'GRAVE'

In [132]:
#verificar alteraçao
df.loc[df.ocorrencia_uf == 'SP']

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_uf_bkp
11,40069,40069,GRAVE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,SP
16,39809,39809,GRAVE,SÃO PAULO,SP,****,2010-01-15,15:00:00,0,SP
18,39828,39828,GRAVE,SANTOS,SP,****,2010-01-15,17:45:00,0,SP
26,39847,39847,GRAVE,SOROCABA,SP,****,2010-01-20,13:10:00,0,SP
27,39768,39768,GRAVE,CAMPINAS,SP,****,2010-01-21,20:45:00,0,SP
...,...,...,...,...,...,...,...,...,...,...
5704,79739,79739,GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0,SP
5705,79705,79705,GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0,SP
5715,79718,79718,GRAVE,SÃO PAULO,SP,SBMT,2020-12-11,13:50:00,0,SP
5742,79800,79800,GRAVE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0,SP


##VAMOS REALIZAR A LIMPEZA DESTA COLUNAS.

ocorrencia_uf

(**)

ocorrencia_aerodromo 

(###!)
(####)
(****)
(*****)

ocorrencia_hora

NULL


In [136]:
#mudar atributos da coluna
df.loc[df.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pd.NA

In [137]:
#alterar os dados em todo o dataset
df.replace(['**', '###!', '####', '****', '*****', 'NULL'], pd.NA, inplace=True)

In [140]:
#verificar a quantidade de dados 'NA'
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2348
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
ocorrencia_uf_bkp              1
dtype: int64

In [ ]:
#alterar todos os dados 'na', para o parametro selecionado
#df.fillna(0, inplace=True)

In [ ]:
#Alterar os paremetro selecionado para 'NA'
#df.replace([10], pd.NA, inplace=True)

In [ ]:
#trocar somente o valores daquela coluna
#df.fillna(value={'total_recomendacoes':10}, inplace=True)

In [142]:
#excluir uma coluna
df.drop(['ocorrencia_uf_bkp'], axis=1, inplace=True)

In [ ]:
#remove todos os valores ausente
#df.dropna()


In [ ]:
#excluir uma linha especifica
#df.dropna(subset=['ocorrencia_uf'])

In [ ]:
#exclui linhas duplicadas
#df.drop_duplicates(inplace=True)